<a href="https://colab.research.google.com/github/shreya139/Circuits-in-Stocks/blob/main/DataPrepration_project_p2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importing Required Library

In [1]:
!pip install yfinance
!pip install fredapi
!pip install yfinance pandas numpy scikit-learn
url = 'https://launchpad.net/~mario-mariomedina/+archive/ubuntu/talib/+files'
ext = '0.4.0-oneiric1_amd64.deb -qO'
!wget $url/libta-lib0_$ext libta.deb
!wget $url/ta-lib0-dev_$ext ta.deb
!dpkg -i libta.deb ta.deb
!pip install ta-lib

Selecting previously unselected package libta-lib0.
(Reading database ... 120895 files and directories currently installed.)
Preparing to unpack libta.deb ...
Unpacking libta-lib0 (0.4.0-oneiric1) ...
Selecting previously unselected package ta-lib0-dev.
Preparing to unpack ta.deb ...
Unpacking ta-lib0-dev (0.4.0-oneiric1) ...
Setting up libta-lib0 (0.4.0-oneiric1) ...
Setting up ta-lib0-dev (0.4.0-oneiric1) ...
Processing triggers for man-db (2.10.2-1) ...
Processing triggers for libc-bin (2.35-0ubuntu3.1) ...
/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc.so.2 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc_proxy.so.2 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_0.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_5.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbb.so.12 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind.so.3 is not a symbolic link

     ━━━━━━━━

In [2]:
pip install nsepy

  Preparing metadata (setup.py) ... done
  Created wheel for nsepy: filename=nsepy-0.8-py3-none-any.whl size=36056 sha256=819859cbe04bd4f47cbd1c371b4fda96f75cf121f97b233face8dbb83238e76a
  Stored in directory: /root/.cache/pip/wheels/4c/87/cb/acaf83f625e5fc73e1fe6e2a8e97680c74cd72391850ef5a86
Successfully built nsepy


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gc
from talib import RSI, BBANDS, MACD, EMA, STOCH, ADX
from nsepy import get_history
from datetime import date
import yfinance as yf

In [4]:
import yfinance as yf
yf.pdr_override()
from pandas_datareader import data as pdr

In [5]:
yf.pdr_override() # <== that's all it takes :-)


#this is reference start date before EMA 200 days will fetch null
ref_startdate = date(2008,1,1)

#actual start date and end date from where the Nifty model to be built
startdate = date(2009,1,1)
enddate = date(2022,12,31)

# download dataframe
data = pdr.get_data_yahoo("^NSEI", start=ref_startdate, end=enddate,interval='1d')
data

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2008-01-01,6136.750000,6165.350098,6109.850098,6144.350098,6144.350098,0
2008-01-02,6144.700195,6197.000000,6060.850098,6179.399902,6179.399902,0
2008-01-03,6184.250000,6230.149902,6126.399902,6178.549805,6178.549805,0
2008-01-04,6179.100098,6300.049805,6179.100098,6274.299805,6274.299805,0
2008-01-07,6271.000000,6289.799805,6193.350098,6279.100098,6279.100098,0
...,...,...,...,...,...,...
2022-12-26,17830.400391,18084.099609,17774.250000,18014.599609,18014.599609,176700
2022-12-27,18089.800781,18149.250000,17967.449219,18132.300781,18132.300781,214300
2022-12-28,18084.750000,18173.099609,18068.349609,18122.500000,18122.500000,193900


After retrieving Nifty records, many brokerage firms utilize the TALIB API to compute key technical indicators like Exponential Moving Averages (EMA), Moving Average Convergence Divergence (MACD), Relative Strength Index (RSI -14), and Bollinger Bands values. These calculated indicators serve as the foundation for generating visually informative technical charts of the Nifty index. This practice is commonplace among various brokerage firms and is instrumental in creating insightful and actionable trading charts for Nifty.

In [6]:
#calculating technical indicators such as EMA-200, EMA-100, EMA-50, EMA-21, EMA-5, MACD, RSI-14, Bollinger Bands
EMA_200 = EMA(data['Close'],200)
EMA_100 = EMA(data['Close'],100)
EMA_50 = EMA(data['Close'],50)
EMA_21 = EMA(data['Close'],21)
EMA_5 = EMA(data['Close'],5)
data['EMA-200']=EMA_200
data['EMA-100']=EMA_100
data['EMA-50']=EMA_50
data['EMA-21']=EMA_21
data['EMA-5']=EMA_5
data['MACD'], data['macdEMA'], data['macdHistory'] = MACD(data['Close'], fastperiod=12, slowperiod=26, signalperiod=9)
data['RSI-14']=RSI(data['Close'],timeperiod=14)

In [7]:
print(data.head(4))

                   Open         High          Low        Close    Adj Close  \
Date                                                                          
2008-01-01  6136.750000  6165.350098  6109.850098  6144.350098  6144.350098   
2008-01-02  6144.700195  6197.000000  6060.850098  6179.399902  6179.399902   
2008-01-03  6184.250000  6230.149902  6126.399902  6178.549805  6178.549805   
2008-01-04  6179.100098  6300.049805  6179.100098  6274.299805  6274.299805   

            Volume  EMA-200  EMA-100  EMA-50  EMA-21  EMA-5  MACD  macdEMA  \
Date                                                                         
2008-01-01       0      NaN      NaN     NaN     NaN    NaN   NaN      NaN   
2008-01-02       0      NaN      NaN     NaN     NaN    NaN   NaN      NaN   
2008-01-03       0      NaN      NaN     NaN     NaN    NaN   NaN      NaN   
2008-01-04       0      NaN      NaN     NaN     NaN    NaN   NaN      NaN   

            macdHistory  RSI-14  
Date                  

In [8]:
print(data.tail(4))

                    Open          High           Low         Close  \
Date                                                                 
2022-12-27  18089.800781  18149.250000  17967.449219  18132.300781   
2022-12-28  18084.750000  18173.099609  18068.349609  18122.500000   
2022-12-29  18045.699219  18229.699219  17992.800781  18191.000000   
2022-12-30  18259.099609  18265.250000  18080.300781  18105.300781   

               Adj Close  Volume       EMA-200       EMA-100        EMA-50  \
Date                                                                         
2022-12-27  18132.300781  214300  17435.654416  17840.502471  18166.168781   
2022-12-28  18122.500000  193900  17442.488700  17846.086581  18164.456280   
2022-12-29  18191.000000  281100  17449.936574  17852.916550  18165.497210   
2022-12-30  18105.300781  192000  17456.457611  17857.914257  18163.136566   

                  EMA-21         EMA-5       MACD    macdEMA  macdHistory  \
Date                             

Retrieve historical data for significant variables such as

Brent Crude Price,

USD-INR exchange rate,

London Stock Exchange values (FTSE index),

USA stock exchange prices (Dow index),
  
Hong Kong stock exchange (Hang Seng Index), and

the India Volatility Index (VIX) from www.Investing.com .




Also, collect historical data related to

Foreign Institutional Investors (FII),

specifically focusing on FII future index values and

FII option index values.

In [17]:
# # Create a ticker object for USD/INR
# ticker = yf.Ticker("USDINR=X")

# # Get historical data for the exchange rate
# exchange_rate_data = ticker.history(start=startdate, end=enddate)

# # Reset the index and convert it into a column
# # exchange_rate_data.reset_index(inplace=True)

# # Print the historical exchange rate data
# exchange_rate_data[['Close']].head(2)

,Close
Date,
2009-01-01 00:00:00+00:00,48.064999
2009-01-02 00:00:00+00:00,48.244999


In [10]:
data.index.dtype
# exchange_rate_data['Date'].dtype

dtype('<M8[ns]')

In [ ]:
# pip install investpy

In [24]:

brent_crude_data = yf.download('BZ=F', start=startdate, end=enddate)

# Get historical data for the exchange rate
exchange_rate_data = yf.download('USDINR=X', start=startdate, end=enddate)

ftse_data = yf.download('^FTSE', start=startdate, end=enddate)

# Collect data for USA stock exchange (Dow index)
dow_data = yf.download('^DJI', start=startdate, end=enddate)

# Collect data for Hong Kong stock exchange (Hang Seng Index)
hang_seng_data = yf.download('^HSI', start=startdate, end=enddate)

# Collect data for India Volatility Index (VIX)
vix_data = yf.download('^VIX', start=startdate, end=enddate)

# Collect historical data for Foreign Institutional Investors (FII) in India
fii_data = yf.download('FII', start=startdate, end=enddate,interval='1d')
#From September 2021, Yahoo Finance and similar free data sources do not
#provide direct access to Foreign Institutional Investors (FII) data for
# the Indian market. FII data is typically not publicly available for free


# Print or use the data as needed
print("Brent Crude")
print(brent_crude_data.head())
print("USD-INR exchange rate,")
print(exchange_rate_data.head())
print("Ftse data")
print(ftse_data.head())
print("Dow data")
print(dow_data.head())
print("Hang seng data")
print(hang_seng_data.head())
print("Vix data")
print(vix_data.head())
print("Fii data")
print(fii_data.head())


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['FII']: Exception('%ticker%: No timezone found, symbol may be delisted')



Brent Crude
                 Open       High        Low      Close  Adj Close  Volume
Date                                                                     
2009-01-02  46.910000  46.910000  46.910000  46.910000  46.910000      50
2009-01-05  46.400002  49.980000  46.400002  49.619999  49.619999     560
2009-01-06  51.779999  52.130001  49.599998  50.529999  50.529999     203
2009-01-07  50.430000  50.549999  45.730000  45.860001  45.860001     244
2009-01-08  45.810001  46.340000  43.799999  44.669998  44.669998     370
USD-INR exchange rate,
                 Open       High        Low      Close  Adj Close  Volume
Date                                                                     
2009-01-01  48.064999  48.825001  48.064999  48.064999  48.064999       0
2009-01-02  48.064999  49.075001  48.064999  48.244999  48.244999       0
2009-01-05  48.799999  48.799999  48.195999  48.229000  48.229000       0
2009-01-06  48.500000  48.759998  48.450001  48.505001  48.505001       0
20

In [25]:
data['Brent_Crude_Price'] = brent_crude_data['Adj Close']
data['USD_INR_Exchange_Rate'] = exchange_rate_data['Adj Close']
data['FTSE_Index'] = ftse_data['Adj Close']
data['Dow_Index'] = dow_data['Adj Close']
data['Hang_Seng_Index'] = hang_seng_data['Adj Close']
data['India_Vix'] = vix_data['Adj Close']

<ipython-input-25-6d99e930f57a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Brent_Crude_Price'] = brent_crude_data['Adj Close']
<ipython-input-25-6d99e930f57a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['USD_INR_Exchange_Rate'] = exchange_rate_data['Adj Close']
<ipython-input-25-6d99e930f57a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:

In [26]:
data

,Open,High,Low,Close,Adj Close,Volume,EMA-200,EMA-100,EMA-50,EMA-21,...,MACD,macdEMA,macdHistory,RSI-14,Brent_Crude_Price,USD_INR_Exchange_Rate,FTSE_Index,Dow_Index,Hang_Seng_Index,India_Vix
Date,,,,,,,,,,,,,,,,,,,,,
2009-01-02,3034.600098,3079.850098,3021.800049,3046.750000,3046.750000,0,4011.815440,3401.533391,3050.956038,2935.980315,...,21.060396,7.767979,13.292417,56.299504,46.910000,48.244999,4561.799805,9034.690430,15042.809570,39.189999
2009-01-05,3058.750000,3131.949951,3056.449951,3121.449951,3121.449951,0,4002.956082,3395.987185,3053.720505,2952.841191,...,32.493886,12.713160,19.780726,59.812761,49.619999,48.229000,4579.600098,8952.889648,15563.309570,39.080002
2009-01-06,3121.500000,3141.800049,3056.100098,3112.800049,3112.800049,0,3994.098808,3390.379519,3056.037350,2967.382905,...,40.391428,18.248814,22.142614,59.219070,50.529999,48.505001,4638.899902,9015.099609,15509.509766,38.560001
2009-01-07,3112.800049,3147.199951,2888.199951,2920.399902,2920.399902,0,3983.415236,3381.072992,3050.718234,2963.111723,...,30.770499,20.753151,10.017348,47.843596,45.860001,48.550999,4507.500000,8769.700195,14987.459961,43.389999
2009-01-09,2919.949951,2929.850098,2810.250000,2873.000000,2873.000000,0,3972.366329,3371.012140,3043.748892,2954.919748,...,19.100885,20.422698,-1.321813,45.523528,44.419998,48.467999,4448.500000,8599.179688,14377.440430,42.820000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-26,17830.400391,18084.099609,17774.250000,18014.599609,18014.599609,176700,17428.652945,17834.607556,18167.551148,18316.482180,...,-56.840691,38.814659,-95.655350,40.672810,NaN,82.604103,NaN,NaN,NaN,NaN
2022-12-27,18089.800781,18149.250000,17967.449219,18132.300781,18132.300781,214300,17435.654416,17840.502471,18166.168781,18299.738416,...,-62.369159,18.577895,-80.947054,44.800308,84.330002,82.902901,NaN,33241.558594,NaN,21.650000
2022-12-28,18084.750000,18173.099609,18068.349609,18122.500000,18122.500000,193900,17442.488700,17846.086581,18164.456280,18283.625833,...,-66.771649,1.507986,-68.279635,44.522543,83.260002,82.776100,7497.200195,32875.710938,19898.910156,22.139999
